In [32]:
# pip install pytorchcv


In [33]:
# pip install json5 imgaug numpy torch


In [ ]:
import os
import cv2
import torch
import numpy as np
from torch.utils.data import Dataset
from torchvision.transforms import transforms
from glob import glob

EMOTION_DICT = {
    "angry": 0,
    "disgust": 1,
    "fear": 2,
    "happy": 3,
    "sad": 4,
    "surprise": 5,
    "neutral": 6,
}


class FER2013Folders(Dataset):
    def __init__(self, data_path, stage="train", image_size=48, transform=None):
        self.data_path = os.path.join(data_path, stage)
        self.image_size = image_size

        # Coletar todos os caminhos de imagem e seus rótulos
        self.image_paths = []
        self.labels = []

        for emotion, label in EMOTION_DICT.items():
            emotion_folder = os.path.join(self.data_path, emotion)
            if not os.path.exists(emotion_folder):
                continue  # Ignorar classes que não existam na pasta
            images = glob(os.path.join(emotion_folder, "*.png")) + \
                     glob(os.path.join(emotion_folder, "*.jpg"))
            self.image_paths.extend(images)
            self.labels.extend([label] * len(images))

        # Use o transform fornecido, ou o transform padrão
        self.transform = transform or transforms.Compose(
            [
                transforms.ToPILImage(),
                transforms.ToTensor(),
                transforms.Normalize([0.5], [0.5]),  # Ajuste para imagens em escala de cinza
            ]
        )

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]

        # Carregar a imagem em escala de cinza
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            raise ValueError(f"Erro ao carregar a imagem: {image_path}")

        # Redimensionar a imagem para o tamanho esperado
        image = cv2.resize(image, (self.image_size, self.image_size))

        # Converter para formato de tensor
        image = np.expand_dims(image, axis=-1)  # Adicionar canal
        image = self.transform(image)

        return image, label


def fer2013_from_folders(data_path, stage="train", image_size=48, transform=None):
    """
    Função para inicializar o dataset a partir de pastas.

    Parameters:
    -----------
    data_path : str
        Caminho para o diretório contendo as imagens.
    stage : str
        Estágio do dataset ('train', 'val', 'test').
    image_size : int
        Tamanho da imagem.
    transform : torchvision.transforms.Compose
        Transformações a serem aplicadas às imagens.

    Returns:
    --------
    FER2013Folders
        Uma instância do dataset.
    """
    return FER2013Folders(data_path, stage, image_size, transform)




In [34]:
import json
import os
import random
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

import imgaug
import numpy as np
import torch
import torch.multiprocessing as mp

seed = 1234
random.seed(seed)
imgaug.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
from torchvision import transforms

import models
from models import segmentation

transform = transforms.Compose([
    transforms.ToTensor(),  # Converte a imagem para tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normaliza para [-1, 1]
])



def main(config_path):
    """
    This is the main function to make the training up

    Parameters:
    -----------
    config_path : srt
        path to config file
    """
    # load configs and set random seed
    configs = json.load(open(config_path))
    configs["cwd"] = os.getcwd()

    # load model and data_loader
    model = get_model(configs)

    train_set, val_set, test_set = get_dataset(configs)

    # init trainer and make a training
    # from trainers.fer2013_trainer import FER2013Trainer
    from trainers.tta_trainer import FER2013Trainer

    # from trainers.centerloss_trainer import FER2013Trainer
    trainer = FER2013Trainer(model, train_set, val_set, test_set, configs)

    if configs["distributed"] == 1:
        ngpus = torch.cuda.device_count()
        mp.spawn(trainer.train, nprocs=ngpus, args=())
    else:
        trainer.train()


def get_model(configs):
    """
    This function get raw models from models package

    Parameters:
    ------------
    configs : dict
        configs dictionary
    """
    try:
        return models.__dict__[configs["arch"]]
    except KeyError:
        return segmentation.__dict__[configs["arch"]]

def get_model2(configs):
    """
    This function gets the model from the models package.

    Parameters:
    ------------
    configs : dict
        Configuration dictionary.
    """
    try:
        # Initialize the model with configurations
        model = models.__dict__[configs["arch"]](num_classes=configs["num_classes"])
        return model
    except KeyError:
        raise KeyError(f"The architecture '{configs['arch']}' is not defined in the models package.")


def get_dataset(configs):
    """
    This function gets raw datasets for training, validation, and testing.

    Parameters:
    -----------
    configs : dict
        Configuration dictionary.

    Returns:
    --------
    tuple
        train_set, val_set, test_set datasets.
    """
    from torchvision.transforms import transforms

    # Define the transformations
    transform = transforms.Compose(
        [
            transforms.ToPILImage(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
            transforms.RandomAffine(degrees=20, translate=(0.1, 0.1), scale=(0.8, 1.2)),
             
        ]
    )

    train_set = fer2013_from_folders("train", configs)
    val_set = fer2013_from_folders("val", configs)
    test_set = fer2013_from_folders("test", configs, tta=True, tta_size=10)

    return train_set, val_set, test_set


if __name__ == "__main__":
    main("./configs/fer2013_config.json")


Start trainer..
{'data_path': 'data/', 'image_size': 224, 'in_channels': 3, 'num_classes': 7, 'arch': 'resnet34', 'lr': 0.0001, 'weighted_loss': 0, 'momentum': 0.9, 'weight_decay': 0.001, 'distributed': 0, 'batch_size': 48, 'num_workers': 6, 'device': 'cuda:0', 'max_epoch_num': 50, 'max_plateau_count': 8, 'plateau_patience': 2, 'steplr': 50, 'log_dir': 'log', 'checkpoint_dir': 'checkpoint', 'model_name': 'test', 'best_model_name': 'best_modelv2.pth', 'last_model_name': 'last_modelv2.pth', 'cwd': 'f:\\Git\\ResidualMaskingNetwork'}


c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


No checkpoint found, starting from scratch


Last model saved to f:\Git\ResidualMaskingNetwork\checkpoint\last_modelv2.pth

E001  1.305/1.065/1.065 50.054/59.535/59.535 | p00  Time 0:04:25



Last model saved to f:\Git\ResidualMaskingNetwork\checkpoint\last_modelv2.pth

E002  0.998/0.977/0.977 62.295/63.615/63.615 | p00  Time 0:08:48



...


E026  0.221/1.057/1.051 92.621/70.623/70.976 | p06  Time 1:54:07




E027  0.215/1.072/1.051 92.833/70.893/70.976 | p07  Time 1:58:30




E028  0.215/1.063/1.051 92.903/70.559/70.976 | p08  Time 2:02:52




E029  0.212/1.080/1.051 93.111/70.440/70.976 | p09  Time 2:07:14

Calc acc on private test with tta..


Traceback (most recent call last):      
  File "f:\Git\ResidualMaskingNetwork\trainers\tta_trainer.py", line 341, in train
    self._test_acc = self._calc_acc_on_private_test_with_tta()
  File "f:\Git\ResidualMaskingNetwork\trainers\tta_trainer.py", line 303, in _calc_acc_on_private_test_with_tta
    outputs = self._model(images)
  File "c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\module.py", line 1553, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\module.py", line 1562, in _call_impl
    return forward_call(*args, **kwargs)
  File "f:\Git\ResidualMaskingNetwork\models\resnet.py", line 269, in forward
    x = self.conv1(x)
  File "c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\module.py", line 1553, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "c:\Users\

In [35]:
# import torch
# import numpy as np
# from sklearn.metrics import confusion_matrix, classification_report
# import matplotlib.pyplot as plt
# import seaborn as sns
# from utils.datasets.fer2013dataset import fer2013
# from tqdm import tqdm

# def test_model(model, test_set, configs):
#     model.eval()
#     device = torch.device(configs["device"])
#     model.to(device)

#     test_loader = torch.utils.data.DataLoader(
#         test_set, batch_size=1, shuffle=False, num_workers=configs["num_workers"]
#     )

#     all_preds = []
#     all_labels = []

#     with torch.no_grad():
#         for images, labels in tqdm(test_loader, desc="Testing"):
#             if len(images.shape) == 5:  # Verifica se o formato é [batch, tta_size, C, H, W]
#                 batch_size, tta_size, C, H, W = images.size()
#                 images = images.view(-1, C, H, W).to(device)  # Reorganiza como [batch * tta_size, C, H, W]
#                 outputs = model(images)
#                 outputs = outputs.view(batch_size, tta_size, -1)  # Volta para [batch, tta_size, num_classes]
#                 outputs = torch.mean(outputs, dim=1)  # Faz a média ao longo do TTA
#             else:
#                 images = images.to(device)
#                 outputs = model(images)

#             labels = labels.to(device)
#             _, preds = torch.max(outputs, 1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())

#         cm = confusion_matrix(all_labels, all_preds)
#         cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]  # Mantendo os valores entre 0 e 1

#         # Relatório de classificação
#         print("Classification Report:")
#         print(classification_report(all_labels, all_preds))

#         # Configurando o gráfico
#         plt.figure(figsize=(10, 8))
#         sns.heatmap(cm_percentage, annot=True, fmt=".2f", cmap="Blues", 
#                     xticklabels=test_set._emotions.columns, 
#                     yticklabels=test_set._emotions.columns)

#         plt.xlabel("Predicted")
#         plt.ylabel("True")
#         plt.title("Residual Masking Network")
#         plt.show()
# if __name__ == "__main__":
#     # Load configs
#     configs = json.load(open("./configs/fer2013_config.json"))
#     configs["cwd"] = os.getcwd()

#     # Load model and dataset
#     checkpoint_path = os.path.join(configs["checkpoint_dir"], f"last_model.pth")
#     print(f"Loading model from {checkpoint_path}")
#     checkpoint = torch.load(checkpoint_path)

# # Verifique as chaves do checkpoint
#     print("Keys in checkpoint:", checkpoint.keys())

#     # Carregar o estado do modelo
#     state_dict = checkpoint["net"]  # Use a chave correta para os pesos do modelo
#     model = get_model2(configs)
#     model.load_state_dict(state_dict)

#     # Load private dataset
#     _, _, test_set = get_dataset(configs)

#     # Test model
#     test_model(model, test_set, configs)

modelo emotenet

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels // reduction, in_channels, bias=False),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)
    
    
class ResidualBlock(nn.Module):
    def __init__(self, in_ch, out_ch, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_ch)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_ch != out_ch:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=stride, padding=0),
                nn.BatchNorm2d(out_ch)
            )
            
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out
    
    
class ResEmoteNet(nn.Module):
    def __init__(self):
        super(ResEmoteNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.relu = nn.ReLU(inplace=True)
        self.se = SEBlock(256)
        
        self.res_block1 = ResidualBlock(256, 512, stride=2)
        self.res_block2 = ResidualBlock(512, 1024, stride=2)
        self.res_block3 = ResidualBlock(1024, 2048, stride=2)
        
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.5)
        self.fc4 = nn.Linear(256, 7)
        
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2)
        x = self.se(x)
        
        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.res_block3(x)
        
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = F.relu(self.fc3(x))
        x = self.dropout2(x)
        x = self.fc4(x)
        return x

In [37]:
import os
import torch
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset

# Dataset class for the custom dataset
class Four4All(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.labels = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        img_name = os.path.join(self.img_dir, self.labels.iloc[idx, 0])
        image = Image.open(img_name)
        label = self.labels.iloc[idx, 1]
        
        if self.transform:
            image = self.transform(image)
            
        return image, label


In [38]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

from torch.utils.data import DataLoader
from torchvision import transforms
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo utilizado: {device}")

# Transform the dataset
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Load the dataset
train_dataset = Four4All(csv_file='data2/fer/train_labels.csv',
                         img_dir='data2/fer/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
train_image, train_label = next(iter(train_loader))


val_dataset = Four4All(csv_file='data2/fer/val_labels.csv', 
                       img_dir='data2/fer/val/', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)
val_image, val_label = next(iter(val_loader))


test_dataset = Four4All(csv_file='data2/fer/test_labels.csv', 
                        img_dir='data2/fer/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
test_image, test_label = next(iter(test_loader))



print(f"Train batch: Image shape {train_image.shape}, Label shape {train_label.shape}")
print(f"Validation batch: Image shape {val_image.shape}, Label shape {val_label.shape}")
print(f"Test batch: Image shape {test_image.shape}, Label shape {test_label.shape}")


# Load the model
model = ResEmoteNet().to(device)


# Print the number of parameters
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')


# Hyperparameters
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)

patience = 15
best_val_acc = 0
patience_counter = 0
epoch_counter = 0

num_epochs = 80

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
test_losses = []
test_accuracies = []

# Verificar se existe checkpoint anterior e carregar o modelo
last_checkpoint_path = './checkpoint/last_emotenet.pth'
best_checkpoint_path = './checkpoint/best_emotenet.pth'
best_val_acc = 0
start_epoch = 0
if os.path.exists(last_checkpoint_path):
    print("Carregando checkpoint anterior...")
    checkpoint = torch.load(last_checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    best_val_acc = checkpoint['best_val_acc']
    start_epoch = checkpoint['epoch']  # Recupera a época salva
    print(f"Checkpoint carregado com validação de melhor acurácia: {best_val_acc:.4f}")
    print(f"Retomando do Epoch {start_epoch}")

# Iniciar o treino
for epoch in range(start_epoch,num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for data in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = correct / total
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    model.eval()
    test_running_loss = 0.0
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

    test_loss = test_running_loss / len(test_loader)
    test_acc = test_correct / test_total
    test_losses.append(test_loss)
    test_accuracies.append(test_acc)

    val_running_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss = val_running_loss / len(val_loader)
    val_acc = val_correct / val_total
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}, "
          f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}, "
          f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

    # Salvar o checkpoint da última época
    torch.save({
    'epoch': epoch + 1,  # Salvar a próxima época
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'best_val_acc': best_val_acc
    }, last_checkpoint_path)


    # Salvar o melhor modelo
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), best_checkpoint_path)
        print("Melhor modelo salvo com nova acurácia de validação: {:.4f}".format(best_val_acc))

    epoch_counter += 1
    if patience_counter > patience:
        print("Stopping early due to lack of improvement in validation accuracy.")
        break
df = pd.DataFrame({
    'Epoch': range(1, epoch_counter+1),
    'Train Loss': train_losses,
    'Test Loss': test_losses,
    'Validation Loss': val_losses,
    'Train Accuracy': train_accuracies,
    'Test Accuracy': test_accuracies,
    'Validation Accuracy': val_accuracies
})
df.to_csv('result_four4all.csv', index=False)

Dispositivo utilizado: cuda:0
Train batch: Image shape torch.Size([16, 3, 64, 64]), Label shape torch.Size([16])
Validation batch: Image shape torch.Size([16, 3, 64, 64]), Label shape torch.Size([16])
Test batch: Image shape torch.Size([16, 3, 64, 64]), Label shape torch.Size([16])
80,238,599 total parameters.
Carregando checkpoint anterior...
Checkpoint carregado com validação de melhor acurácia: 0.7712
Retomando do Epoch 23


Epoch 24/80: 100%|██████████| 2264/2264 [08:31<00:00,  4.42it/s]


Epoch 24, Train Loss: 0.4912, Train Accuracy: 0.8313, Test Loss: 0.5770, Test Accuracy: 0.7952, Validation Loss: 0.6280, Validation Accuracy: 0.7843
Melhor modelo salvo com nova acurácia de validação: 0.7843


Epoch 25/80: 100%|██████████| 2264/2264 [04:41<00:00,  8.05it/s]


Epoch 25, Train Loss: 0.4656, Train Accuracy: 0.8382, Test Loss: 0.5449, Test Accuracy: 0.8113, Validation Loss: 0.5739, Validation Accuracy: 0.8149
Melhor modelo salvo com nova acurácia de validação: 0.8149


...

Epoch 78, Train Loss: 0.0596, Train Accuracy: 0.9801, Test Loss: 0.4070, Test Accuracy: 0.9025, Validation Loss: 0.4554, Validation Accuracy: 0.8905


Epoch 79/80: 100%|██████████| 2264/2264 [04:54<00:00,  7.68it/s]


Epoch 79, Train Loss: 0.0552, Train Accuracy: 0.9815, Test Loss: 0.4317, Test Accuracy: 0.8974, Validation Loss: 0.4716, Validation Accuracy: 0.8983


Epoch 80/80: 100%|██████████| 2264/2264 [04:41<00:00,  8.04it/s]


Epoch 80, Train Loss: 0.0621, Train Accuracy: 0.9803, Test Loss: 0.4391, Test Accuracy: 0.8815, Validation Loss: 0.4201, Validation Accuracy: 0.8916


My cnn normal

In [39]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm

# Configuração dos hiperparâmetros
specific_hyperparams = {
    'weight_decay': 0.0001,
    'patience': 5,
    'momentum': 0.9,
    'lr': 0.01,
    'epochs': 50,
    'batch_size': 64,
}

# Dataset e DataLoaders
class DataLoaderSetup:
    def __init__(self, dataset_path, image_size=224, batch_size=64):
        self.dataset_path = dataset_path
        self.image_size = image_size
        self.batch_size = batch_size

    def load_data(self):
        transform = {
            'treino': transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomResizedCrop(self.image_size),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
            'validacao': transforms.Compose([
                transforms.Resize(self.image_size),
                transforms.CenterCrop(self.image_size),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        }

        data = {
            'treino': datasets.ImageFolder(root=os.path.join(self.dataset_path, 'treino'), transform=transform['treino']),
            'validacao': datasets.ImageFolder(root=os.path.join(self.dataset_path, 'validacao'), transform=transform['validacao']),
        }

        data_loader_treino = DataLoader(data['treino'], batch_size=self.batch_size, shuffle=True)
        data_loader_validacao = DataLoader(data['validacao'], batch_size=self.batch_size, shuffle=False)

        return data_loader_treino, data_loader_validacao, len(data['treino']), len(data['validacao']), len(data['treino'].classes)

# Modelo Customizado
class CustomResNet50:
    def __init__(self, num_classes):
        self.model = models.resnet50(pretrained=True)
        for param in self.model.parameters():
            param.requires_grad = False  # Congelar todas as camadas

        for param in self.model.layer4.parameters():  # Descongelar o bloco "layer4"
            param.requires_grad = True

        num_features = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def get_model(self):
        return self.model

# Treinamento
class Trainer:
    def __init__(self, model, data_loader_treino, data_loader_validacao, device, hyperparams):
        self.model = model
        self.data_loader_treino = data_loader_treino
        self.data_loader_validacao = data_loader_validacao
        self.device = device
        self.hyperparams = hyperparams

        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.SGD(self.model.parameters(), lr=hyperparams['lr'], momentum=hyperparams['momentum'], weight_decay=hyperparams['weight_decay'])
        self.scheduler = ReduceLROnPlateau(self.optimizer, mode='min', factor=0.1, patience=3, verbose=True)
        self.patience = hyperparams['patience']
        self.best_accuracy = 0.0

        # Diretório para salvar checkpoints
        self.checkpoint_dir = './checkpoint'
        os.makedirs(self.checkpoint_dir, exist_ok=True)

        # Caminho do último checkpoint
        self.last_checkpoint_path = os.path.join(self.checkpoint_dir, "my_resnet_last.pth")

        # Carregar o checkpoint se existir
        self.start_epoch = 0
        if os.path.exists(self.last_checkpoint_path):
            print("Checkpoint encontrado. Carregando...")
            checkpoint = torch.load(self.last_checkpoint_path)
            self.model.load_state_dict(checkpoint['model_state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
            self.start_epoch = checkpoint['epoch'] + 1
            self.best_accuracy = checkpoint['best_accuracy']
            print(f"Retomando do epoch {self.start_epoch}, Melhor Acurácia: {self.best_accuracy:.4f}")
        else:
            print("Nenhum checkpoint encontrado. Iniciando do zero.")

    def train_and_evaluate(self):
        early_stop_counter = 0

        for epoch in range(self.start_epoch, self.hyperparams['epochs']):
            print(f"\nEpoch {epoch + 1}/{self.hyperparams['epochs']}")

            # Treinamento
            self.model.train()
            train_loss, train_correct, train_total = 0.0, 0, 0
            for inputs, labels in tqdm(self.data_loader_treino, desc="Treinando", leave=False):
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                self.optimizer.zero_grad()

                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

                train_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                train_correct += (preds == labels).sum().item()
                train_total += labels.size(0)

            train_accuracy = train_correct / train_total
            print(f"Train Loss: {train_loss / train_total:.4f}, Accuracy: {train_accuracy * 100:.2f}%")

            # Validação
            self.model.eval()
            val_loss, val_correct, val_total = 0.0, 0, 0
            with torch.no_grad():
                for inputs, labels in tqdm(self.data_loader_validacao, desc="Validando", leave=False):
                    inputs, labels = inputs.to(self.device), labels.to(self.device)
                    outputs = self.model(inputs)
                    loss = self.criterion(outputs, labels)

                    val_loss += loss.item() * inputs.size(0)
                    _, preds = torch.max(outputs, 1)
                    val_correct += (preds == labels).sum().item()
                    val_total += labels.size(0)

            val_accuracy = val_correct / val_total
            print(f"Validation Loss: {val_loss / val_total:.4f}, Accuracy: {val_accuracy * 100:.2f}%")

            self.scheduler.step(val_loss)

            # Salvar checkpoint da última época
            torch.save({
                'epoch': epoch,
                'model_state_dict': self.model.state_dict(),
                'optimizer_state_dict': self.optimizer.state_dict(),
                'scheduler_state_dict': self.scheduler.state_dict(),
                'best_accuracy': self.best_accuracy
            }, self.last_checkpoint_path)

            # Salvar o melhor modelo
            if val_accuracy > self.best_accuracy:
                self.best_accuracy = val_accuracy
                torch.save(self.model.state_dict(), os.path.join(self.checkpoint_dir, "my_best_resnet.pth"))
                print("Melhor modelo salvo.")
                early_stop_counter = 0
            else:
                early_stop_counter += 1

            if early_stop_counter >= self.patience:
                print("Parando o treinamento devido ao early stopping.")
                break

        print(f"Melhor Acurácia de Validação: {self.best_accuracy:.4f}")

# Configuração do dataset
dataset_path = './data2/fer/fer2/'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_loader_setup = DataLoaderSetup(dataset_path, image_size=224, batch_size=specific_hyperparams['batch_size'])
data_loader_treino, data_loader_validacao, _, _, num_classes = data_loader_setup.load_data()

# Inicializar e treinar o modelo
modelo = CustomResNet50(num_classes).get_model().to(device)
trainer = Trainer(modelo, data_loader_treino, data_loader_validacao, device, specific_hyperparams)
trainer.train_and_evaluate()


c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Checkpoint encontrado. Carregando...
Retomando do epoch 1, Melhor Acurácia: 0.0000

Epoch 2/50


Train Loss: 1.3940, Accuracy: 46.45%


Validation Loss: 1.1651, Accuracy: 56.42%
Melhor modelo salvo.

Epoch 3/50


Train Loss: 1.3330, Accuracy: 49.29%


...

Train Loss: 0.9209, Accuracy: 65.91%


Validation Loss: 0.7341, Accuracy: 73.95%

Epoch 38/50


Train Loss: 0.9284, Accuracy: 65.57%


Validation Loss: 0.7235, Accuracy: 73.81%

Epoch 39/50


KeyboardInterrupt: 